In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from transformers import BertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import numpy as np

from data_utils import HateDataset
from utils import initialize_seeds

In [ ]:
initialize_seeds()

In [ ]:
# Load datasets
training_data = pd.read_pickle('./hatecheck-experiments/Data/Clean Training Data/training_data_binary.pkl')

df_raw = {}

# write to dict
for dataset in training_data:
    df_raw[dataset] = training_data[dataset].copy()

In [ ]:
# Split each dataset into training and validation set
df_train, df_valtest, df_val, df_test = {}, {}, {}, {}

for dataset in df_raw:
    df_train[dataset], df_valtest[dataset] = train_test_split(df_raw[dataset], test_size=0.2, stratify=df_raw[dataset].label, random_state=123)
    df_val[dataset], df_test[dataset] = train_test_split(df_valtest[dataset], test_size=0.5, stratify=df_valtest[dataset].label, random_state=123)

In [ ]:
# Split up text and label columns in dataframes into series for each dataset
train_texts, val_texts, test_texts, train_labels, val_labels, test_labels = {}, {}, {}, {}, {}, {}

for dataset in df_raw:
    train_texts[dataset] = df_train[dataset].text.astype("string").tolist()
    val_texts[dataset] = df_val[dataset].text.astype("string").tolist()
    test_texts[dataset] = df_test[dataset].text.astype("string").tolist()
    
    train_labels[dataset] = df_train[dataset].label.tolist()
    val_labels[dataset] = df_val[dataset].label.tolist()
    test_labels[dataset] = df_test[dataset].label.tolist()

In [ ]:
# compute class weights based on training data label distribution
class_weights = {}

for dataset in df_raw:
    class_weights[dataset] = compute_class_weight('balanced', classes = np.unique(train_labels[dataset]), y = train_labels[dataset])

class_weights

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
num_added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[USER]','[EMOJI]','[URL]']})

# Tokenize text series for each dataset
train_encodings, val_encodings, test_encodings = {}, {}, {}

for dataset in df_raw:
    train_encodings[dataset] = tokenizer(train_texts[dataset], truncation=True, padding=True)
    val_encodings[dataset] = tokenizer(val_texts[dataset], truncation=True, padding=True)
    test_encodings[dataset] = tokenizer(test_texts[dataset], truncation=True, padding=True)

In [ ]:
train_dataset, val_dataset, test_dataset = {}, {}, {}
    
for dataset in df_raw:
    train_dataset[dataset] = HateDataset(train_encodings[dataset], train_labels[dataset])
    val_dataset[dataset] = HateDataset(val_encodings[dataset], val_labels[dataset])
    test_dataset[dataset] = HateDataset(test_encodings[dataset], test_labels[dataset])

In [ ]:
filename = "./data/targetData.pt"
torch.save([train_dataset, val_dataset, test_dataset], filename)